In [1]:
print('teste')

teste


In [28]:
base_dir = "/ztemp/ml/processing"

In [ ]:
#mkdir -p /opt/ml/processing/validation/

In [9]:
#sudo mkdir -p /ztemp/
#sudo mkdir -p /ztemp/ml/processing/train/
#sudo mkdir -p /ztemp/ml/processing/validation/
#sudo mkdir -p /ztemp/ml/processing/test/
#sudo mkdir -p /ztemp/ml/processing/modeldata/
#sudo chmod -R 777 /ztemp/

mkdir: cannot create directory ‘/opt/ml/processing’: Permission denied
mkdir: cannot create directory ‘/opt/ml/processing’: Permission denied
mkdir: cannot create directory ‘/opt/ml/processing’: Permission denied
mkdir: cannot create directory ‘/opt/ml/processing’: Permission denied


In [ ]:
#!pip install mlflow==2.13.2 sagemaker-mlflow==0.1.0

In [86]:
import datetime

In [29]:
import boto3
import numpy as np
import pandas as pd

In [124]:
import sagemaker

In [87]:
datenow = datetime.datetime.now().strftime("%Y-%m-%d-%H%M")
print(datenow)

2024-10-07-2315


In [49]:
s3 = boto3.resource("s3")
bucket = 'z-tmp-tfc'
#key = f"s3://{bucket}/data/modeldata/fulldataset.csv"
key = f"data/modeldata/fulldataset.csv"
fn = f"{base_dir}/modeldata/fulldataset.csv"

In [50]:
print(key)

data/modeldata/fulldataset.csv


In [51]:
s3.Bucket(bucket).download_file(key, fn)

In [89]:
import mlflow

from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [90]:
from sklearn.preprocessing import StandardScaler


In [92]:
mlflow.set_tracking_uri('arn:aws:sagemaker:us-east-1:828238096174:mlflow-tracking-server/apg-mlflow-sage')
mlflow.set_experiment("tfc-exp-lr-heart2")

2024/10/07 23:19:35 INFO mlflow.tracking.fluent: Experiment with name 'tfc-exp-lr-heart2' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://my-unique-bucket-name-test-zzz/art/2', creation_time=1728343175072, experiment_id='2', last_update_time=1728343175072, lifecycle_stage='active', name='tfc-exp-lr-heart2', tags={}>

In [93]:
# Load the Iris dataset
#base_dir = "/opt/ml/processing"
fn = f"{base_dir}/modeldata/fulldataset.csv"
df = pd.read_csv(fn)
#X, y = pd.read_csv(fn)
#X = pd.read_csv(fn).drop('num_bins', axis=1)
#print(df)
X = df.drop('num_bins', axis=1)
y = df.num_bins


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [94]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [95]:
st_x = StandardScaler()
X_train = st_x.fit_transform(X_train)
X_test = st_x.transform(X_test)

In [96]:
print('X_train dimension= ', X_train.shape)
print('X_test dimension= ', X_test.shape)
print('y_train dimension= ', y_train.shape)
print('y_train dimension= ', y_test.shape)

X_train dimension=  (733, 14)
X_test dimension=  (184, 14)
y_train dimension=  (733,)
y_train dimension=  (184,)


In [97]:
print(X_train)

[[-1.13501564  0.52419711 -1.07714551 ...  1.13407085  0.71880339
  -0.65550334]
 [ 1.21245931  0.52419711  1.60839075 ... -0.53851252  0.71880339
   0.93550311]
 [-2.62886334  0.52419711 -0.18196676 ...  1.13407085 -0.61170894
  -0.65550334]
 ...
 [ 1.42586613  0.52419711 -1.07714551 ... -0.53851252 -0.61170894
  -0.65550334]
 [ 1.95938316  0.52419711  1.60839075 ... -0.53851252  0.71880339
   0.93550311]
 [-1.77523608 -1.90767935 -0.18196676 ...  1.13407085 -0.61170894
  -0.65550334]]


In [98]:
# Define the model hyperparameters
params = {"multi_class": "auto", "C": 1, "penalty":"l2", "max_iter": 1000}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
#y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
#accuracy = accuracy_score(y_test, y_pred)


LogisticRegression(C=1, max_iter=1000)

In [99]:
y_pred = lr.predict(X_test)


In [100]:
accuracy = accuracy_score(y_test, y_pred)


In [101]:
print(accuracy)

0.7934782608695652


In [102]:
print(X_test)

[[-1.34842245 -1.90767935 -1.07714551 ...  1.13407085  0.71880339
  -0.65550334]
 [ 1.10575591  0.52419711  1.60839075 ... -0.53851252  3.37982804
   0.93550311]
 [ 1.21245931  0.52419711 -0.18196676 ... -0.53851252 -0.61170894
   0.93550311]
 ...
 [-0.17468498  0.52419711 -1.07714551 ...  1.13407085 -0.61170894
  -0.65550334]
 [ 0.67894228 -1.90767935 -1.07714551 ...  1.13407085 -0.61170894
  -0.65550334]
 [ 0.03872184  0.52419711  1.60839075 ... -0.53851252 -0.61170894
   0.93550311]]


In [103]:
# Start an MLflow run and log parameters, metrics, and model artifacts
with mlflow.start_run() as run:
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for heart data")

    mlflow.set_tag("mlflow.runName", "tfc_run_heart_" + datenow)
    
    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="tfc_mlf_lr_heart",
        signature=signature,
        input_example=X_train,
        registered_model_name="tfc-mlf-lr-hear",
    )

/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'tfc-mlf-lr-hear'.
2024/10/07 23:20:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version t

2024/10/07 23:20:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run tfc_run_heart_2024-10-07-2315 at: https://us-east-1.experiments.sagemaker.aws/#/experiments/2/runs/e8ccf19465eb4419a8065515856e4aac.
2024/10/07 23:20:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/2.


In [106]:
model_uri = f"runs:/{run.info.run_id}/tfc-mlf-lr-hear"

print(model_uri)


runs:/e8ccf19465eb4419a8065515856e4aac/tfc-mlf-lr-heart


In [107]:


mv = mlflow.register_model(model_uri, "tfc-model-lr-hear")

print(f"Name: {mv.name}")
print(f"Version: {mv.version}")

Successfully registered model 'tfc-model-lr-hear'.
2024/10/07 23:23:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tfc-model-lr-hear, version 1


Name: tfc-model-lr-hear
Version: 1


Created version '1' of model 'tfc-model-lr-hear'.


In [118]:
'''
from sagemaker.serve import SchemaBuilder

sklearn_input = X_train.reshape(1, -1)
#sklearn_input = np.array([1.0, 2.0, 3.0, 4.0]).reshape(1, -1)
sklearn_output = 1
sklearn_schema_builder = SchemaBuilder(
    sample_input=sklearn_input,
    sample_output=sklearn_output,
)
'''

In [122]:
print(sklearn_input)

[[-1.13501564  0.52419711 -1.07714551 ...  1.13407085  0.71880339
  -0.65550334]
 [ 1.21245931  0.52419711  1.60839075 ... -0.53851252  0.71880339
   0.93550311]
 [-2.62886334  0.52419711 -0.18196676 ...  1.13407085 -0.61170894
  -0.65550334]
 ...
 [ 1.42586613  0.52419711 -1.07714551 ... -0.53851252 -0.61170894
  -0.65550334]
 [ 1.95938316  0.52419711  1.60839075 ... -0.53851252  0.71880339
   0.93550311]
 [-1.77523608 -1.90767935 -0.18196676 ...  1.13407085 -0.61170894
  -0.65550334]]
[[-1.13501564  0.52419711 -1.07714551 ...  1.13407085 -0.61170894
  -0.65550334]]


In [130]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::828238096174:role/apg-role-sage


AttributeError: 'ModelVersion' object has no attribute 'latest_versions'

In [131]:
model_name = mv.name
model_version = mv.version
model_path = f"models:/{model_name}/{model_version}"
print(model_path)

models:/tfc-model-lr-hear/1


In [143]:
#model_uri = f"runs:/{run.info.run_id}/tfc-mlf-lr-hear"
#print(model_uri)

model_uri = "runs:/{run_id}/{artifact_path}".format(run_id=run.info.run_id, artifact_path='tfc-mlf-lr-hear')
print(model_uri)

runs:/e8ccf19465eb4419a8065515856e4aac/tfc-mlf-lr-hear


In [132]:
mlflow_arn = f"arn:aws:sagemaker:us-east-1:828238096174:mlflow-tracking-server/apg-mlflow-sage"
print(mlflow_arn)

arn:aws:sagemaker:us-east-1:828238096174:mlflow-tracking-server/apg-mlflow-sage


In [201]:
client = mlflow.MlflowClient()
registered_model = client.get_registered_model(name="tfc-mlf-lr-hear")
#model_source_path = registered_model.latest_versions[0].source
model_source_path = f"s3://my-unique-bucket-name-test-zzz/art/z0/e8ccf19465eb4419a8065515856e4aac/artifacts/tfc_mlf_lr_heart"
print(model_source_path)

s3://my-unique-bucket-name-test-zzz/art/z0/e8ccf19465eb4419a8065515856e4aac/artifacts/tfc_mlf_lr_heart


In [199]:
#image_uri_model = f"081325390199.dkr.ecr.us-east-1.amazonaws.com/sagemaker-data-science-311-v1"
#image_uri_model = sagemaker.image_uris.retrieve("linear-learner", "eu-central-1")
#image_uri_model = sagemaker.image_uris.get_base_python_image_uri('us-east-1', py_version='310')
#sagemaker.image_uris.retrieve("sagemaker-data-science-311-v1	", "us-east-1")
#image_uri_model = sagemaker.image_uris.retrieve(framework='pytorch',region='us-east-1',version='2.0.0',py_version='py310',image_scope='inference')
#image_uri_model = sagemaker.image_uris.retrieve(framework='pytorch',region='us-east-1')
#image_uri_model = sagemaker.image_uris.retrieve(framework='sklearn',region='us-east-2',version='1.2-1',image_scope='inference')
#image_uri_model = sagemaker.image_uris.retrieve(framework='pytorch',region='us-east-1',version='2.0.0',py_version='py310',image_scope='inference', instance_type='ml.c5.4xlarge')
#image_uri_model = sagemaker.image_uris.retrieve(framework='sklearn',region='us-east-2',version='1.2-1', py_version='py310',image_scope='inference', instance_type='ml.c5.4xlarge')
image_uri_model = sagemaker.image_uris.retrieve(framework='sklearn',region='us-east-1',version='1.2-1',image_scope='inference')
print(image_uri_model)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3


TypeError: get_training_image_uri() missing 2 required positional arguments: 'region' and 'framework'

In [202]:
from sagemaker.serve import ModelBuilder
from sagemaker.serve.mode.function_pointers import Mode
from sagemaker.serve import SchemaBuilder

#sklearn_input = X_train.reshape(1, -1)
#sklearn_output = 1

sklearn_input = X_test[[0, 1]]
sklearn_output = [1, 2]

my_schema = SchemaBuilder(
    sample_input=sklearn_input, 
    sample_output=sklearn_output
)

model_builder = ModelBuilder(
    name="tfc-mlf-lr-heart",
    mode=Mode.SAGEMAKER_ENDPOINT,
    schema_builder=my_schema,
    role_arn=role,
    image_uri=image_uri_model,
    model_metadata={
        # both model path and tracking server ARN are required if you use an mlflow run ID or mlflow model registry path as input
        "MLFLOW_MODEL_PATH": model_source_path,
        "MLFLOW_TRACKING_ARN": mlflow_arn
    }
)

In [203]:
model = model_builder.build()

ModelBuilder: INFO:     Downloading art/z0/e8ccf19465eb4419a8065515856e4aac/artifacts/tfc_mlf_lr_heart/MLmodel to /tmp/sagemaker/model-builder/73f9e562850b11efa22b6edf4e483307/MLmodel
ModelBuilder: INFO:     Downloading art/z0/e8ccf19465eb4419a8065515856e4aac/artifacts/tfc_mlf_lr_heart/conda.yaml to /tmp/sagemaker/model-builder/73f9e562850b11efa22b6edf4e483307/conda.yaml
ModelBuilder: INFO:     Downloading art/z0/e8ccf19465eb4419a8065515856e4aac/artifacts/tfc_mlf_lr_heart/input_example.json to /tmp/sagemaker/model-builder/73f9e562850b11efa22b6edf4e483307/input_example.json
ModelBuilder: INFO:     Downloading art/z0/e8ccf19465eb4419a8065515856e4aac/artifacts/tfc_mlf_lr_heart/model.pkl to /tmp/sagemaker/model-builder/73f9e562850b11efa22b6edf4e483307/model.pkl
ModelBuilder: INFO:     Downloading art/z0/e8ccf19465eb4419a8065515856e4aac/artifacts/tfc_mlf_lr_heart/python_env.yaml to /tmp/sagemaker/model-builder/73f9e562850b11efa22b6edf4e483307/python_env.yaml
ModelBuilder: INFO:     Download

In [204]:
predictor = model.deploy(
                endpoint_name="tfc-ept-mlf-lr-heart",
                initial_instance_count=1,
                instance_type="ml.m5.large"
            )

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-10-09-01-08-29-089
INFO:sagemaker:Creating endpoint-config with name tfc-ept-mlf-lr-heart
INFO:sagemaker:Creating endpoint with name tfc-ept-mlf-lr-heart


------------!

ModelBuilder: INFO:     Polling lineage artifact for model data in s3://sagemaker-us-east-1-828238096174/sagemaker-scikit-learn-2024-10-09-01-07-59-151/serve.tar.gz
ModelBuilder: INFO:     Adding association with source_arn: arn:aws:sagemaker:us-east-1:828238096174:artifact/934b239ee9160e3ffe7a669155e064a2, destination_arn: arn:aws:sagemaker:us-east-1:828238096174:artifact/578ce46c46312a4e5fd4e433152f6764 and association_type: ContributedTo.
ModelBuilder: DEBUG:     ModelBuilder metrics emitted.


In [288]:
#predictor.predict(sklearn_input)


In [293]:
import io

sagemaker_runtime_client = boto3.client('sagemaker-runtime')

# works as expected:
buffer = io.BytesIO()
np.save(buffer, X_test)
invoke_response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    Body=buffer.getvalue(),
    ContentType='application/x-npy',
    Accept='application/json'
)



In [294]:
print(invoke_response)
print(invoke_response['Body'])

{'ResponseMetadata': {'RequestId': 'd8129576-e111-4cf1-8e31-3acd50e6c183', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd8129576-e111-4cf1-8e31-3acd50e6c183', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Wed, 09 Oct 2024 03:26:55 GMT', 'content-type': 'text/html; charset=utf-8', 'content-length': '552', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'text/html; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f81d5b19780>}


In [246]:
predictions = invoke_response['Body'].read()

In [247]:
print(predictions)

b'[2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 1, 2, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2]'


In [282]:
import codecs

x_string = codecs.decode(predictions, 'utf-8')
print(str(x_string))

[2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 1, 2, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2]


In [285]:
x_string = x_string.replace("[", "")
x_string = x_string.replace("]", "")
print(x_string)

2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 1, 2, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2


In [286]:
dftmp = pd.DataFrame({'num_bins': x_string.split(",")})

In [287]:
print(dftmp)

    num_bins
0          2
1          1
2          2
3          1
4          2
..       ...
179        2
180        2
181        2
182        2
183        2

[184 rows x 1 columns]


In [ ]:
#model.delete_model(model_name=model.name)
#model.delete_endpoint_config(endpoint_config_name=model.endpoint_name)
#model.delete_endpoint(endpoint_name=model.endpoint_name

In [292]:
print(sklearn_input)

[[-1.13501564  0.52419711 -1.07714551 ...  1.13407085 -0.61170894
  -0.65550334]]


In [298]:
print(X_test)

[[-1.34842245 -1.90767935 -1.07714551 ...  1.13407085  0.71880339
  -0.65550334]
 [ 1.10575591  0.52419711  1.60839075 ... -0.53851252  3.37982804
   0.93550311]
 [ 1.21245931  0.52419711 -0.18196676 ... -0.53851252 -0.61170894
   0.93550311]
 ...
 [-0.17468498  0.52419711 -1.07714551 ...  1.13407085 -0.61170894
  -0.65550334]
 [ 0.67894228 -1.90767935 -1.07714551 ...  1.13407085 -0.61170894
  -0.65550334]
 [ 0.03872184  0.52419711  1.60839075 ... -0.53851252 -0.61170894
   0.93550311]]


In [319]:
#sklearn_input = X_train.reshape(1, -1)
type(X_test)

numpy.ndarray

In [326]:
sklearn_input = X_test[[0, 1]] 

In [333]:
sklearn_input = X_test[[0]] 

In [334]:
type(sklearn_input)


numpy.ndarray

In [335]:
# works as expected:
print(sklearn_input)
buffer = io.BytesIO()
np.save(buffer, sklearn_input)
print(buffer)

[[-1.34842245 -1.90767935 -1.07714551  0.23449254 -1.56429513 -0.98320783
  -0.43137471  0.0389366   1.22119053 -0.78619434 -0.7846153   1.13407085
   0.71880339 -0.65550334]]


In [336]:
invoke_response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    Body=buffer.getvalue(),
    ContentType='application/x-npy',
    Accept='application/json'
)


In [337]:
prediction_a = invoke_response['Body'].read()

In [338]:
print(prediction_a)

b'[2]'
